# 🧼 01 - Data Cleaning: Clinical Trials Dataset

In [26]:

import pandas as pd
from pathlib import Path

BASE_DIR = Path.cwd()
DATA_PATH = BASE_DIR / "data" / "clinical_trials_sample.csv"

if not DATA_PATH.exists():  
    raise FileNotFoundError(f"Data file not found at {DATA_PATH}")

df = pd.read_csv(DATA_PATH)
df.head()

,study_id,intervention_name,condition,sponsor,start_date,end_date,phase,status
0,NCT100000,Aspirin,COVID-19,Sanofi,2022-09-20,2025-04-07,Phase 2,Withdrawn
1,NCT100001,Trastuzumab,Alzheimer's Disease,Pfizer,2023-07-01,2024-09-05,NaN,Terminated
2,NCT100002,Donepezil,Hypertension,Pfizer,2021-08-17,2025-05-24,Phase 1,Completed
3,NCT100003,Remdesivir,Breast Cancer,Sanofi,2022-04-15,2025-05-20,Phase 1,Terminated
4,NCT100004,Aspirin,Breast Cancer,Pfizer,2023-09-10,2024-08-28,Phase 4,Completed


## 🔍 Step 1: Inspect Data Structure

In [27]:

df.info()
df.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   study_id           1200 non-null   object
 1   intervention_name  1139 non-null   object
 2   condition          1165 non-null   object
 3   sponsor            1200 non-null   object
 4   start_date         1200 non-null   object
 5   end_date           963 non-null    object
 6   phase              948 non-null    object
 7   status             1200 non-null   object
dtypes: object(8)
memory usage: 75.1+ KB


,study_id,intervention_name,condition,sponsor,start_date,end_date,phase,status
count,1200,1139,1165,1200,1200,963,948,1200
unique,1200,6,6,6,800,335,4,4
top,NCT101183,Salbutamol,Alzheimer's Disease,Sanofi,2021-05-22,2024-09-03,Phase 4,Completed
freq,1,205,211,214,6,7,258,312


## 📊 Step 2: Identify and Handle Missing Values

In [28]:

# Count missing values
missing = df.isnull().sum()
print("Missing Values:\n", missing)

# Fill empty strings with NaN
df.replace('', pd.NA, inplace=True)

# Create a 'completeness_score' column
df['completeness_score'] = df.notna().mean(axis=1)
df[['study_id', 'completeness_score']].head()


Missing Values:
 study_id               0
intervention_name     61
condition             35
sponsor                0
start_date             0
end_date             237
phase                252
status                 0
dtype: int64


,study_id,completeness_score
0,NCT100000,1.000
1,NCT100001,0.875
2,NCT100002,1.000
3,NCT100003,1.000
4,NCT100004,1.000


## 🔁 Step 3: Remove Duplicates and Normalize Case

In [29]:

# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Normalize casing
df['intervention_name'] = df['intervention_name'].str.title()
df['condition'] = df['condition'].str.title()
df['sponsor'] = df['sponsor'].str.title()


## 📅 Step 4: Format Dates

In [30]:

df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')
df['end_date'] = pd.to_datetime(df['end_date'], errors='coerce')

# Calculate duration in days
df['study_duration_days'] = (df['end_date'] - df['start_date']).dt.days
df[['study_id', 'start_date', 'end_date', 'study_duration_days']].head()


,study_id,start_date,end_date,study_duration_days
0,NCT100000,2022-09-20,2025-04-07,930.0
1,NCT100001,2023-07-01,2024-09-05,432.0
2,NCT100002,2021-08-17,2025-05-24,1376.0
3,NCT100003,2022-04-15,2025-05-20,1131.0
4,NCT100004,2023-09-10,2024-08-28,353.0


## 💾 Step 5: Save Cleaned Data

In [35]:

df.to_csv(DATA_PATH.parent / "clinical_trials_cleaned.csv", index=False)
print("Cleaned data saved.")


Cleaned data saved.
